In [2]:
import folium
import requests
import json
import pandas as pd
import numpy as np

## 실습 230907 
### 1. 의료공공기관 현황 주소에서 시도, 군구를 추출한다.
### 2. 서울데이터만 추출
### 3. 군구별 의료기관 수를 구한다.
### 4. 행정구역 서울의 군구별 인구수를 추출한다.
### 5. 서울의 군구별 인구비율 공공의료기관수 비율을 구한다.
### 6. 위 데이터를 활용하여 choropleth map을 작성한다

In [3]:
# 1. 의료공공기관 현황 주소에서 시도, 군구를 추출한다.
hosiptal_csv = pd.read_csv('data/보건복지부_공공보건 의료기관 현황_20161231.csv')
hosiptal_csv_addr = pd.DataFrame(hosiptal_csv['주소'].apply(lambda v : v.split()[:2]).to_list(), columns=['시도', '군구'])

In [4]:
# 2. 서울데이터만 추출
hosiptal_csv_addr['시도'].value_counts() # 서울특별시 16 , 서울시 5
seouls_alias = {'서울시' : '서울특별시'}
hosiptal_csv_addr['시도'] = hosiptal_csv_addr['시도'].apply(lambda x : seouls_alias.get(x, x))
hosiptal_csv_addr['시도'].value_counts() # 서울특별시 21
hospital_in_seoul = hosiptal_csv_addr.loc[hosiptal_csv_addr['시도'] == '서울특별시']


In [5]:

# 3. 군구별 의료기관 수를 구한다.
hospital_count_by_goongoo = hospital_in_seoul['군구'].value_counts()
hospital_count_by_goongoo_df = pd.DataFrame({'행정구역군구' : hospital_count_by_goongoo.index, '총의료기관수' : hospital_count_by_goongoo.values})
hospital_count_by_goongoo_df


,행정구역군구,총의료기관수
0,종로구,4
1,은평구,2
2,중랑구,2
3,강남구,2
4,강동구,1
5,노원구,1
6,강북구,1
7,광진구,1
8,중구,1
9,서초구,1


In [6]:
# 4. 행정구역 서울의 군구별 인구수를 추출한다.
population_csv = pd.read_csv('data/행정구역_시군구_별__성별_인구수_20230717102210.csv')
population_csv_not_total = population_csv.loc[population_csv['행정구역군구'] != '소계']
population_csv_not_total_in_seoul = population_csv_not_total.loc[population_csv_not_total['행정구역시도'] == '서울특별시']
population_csv_not_total_in_seoul_goongoo_total = population_csv_not_total_in_seoul[['행정구역군구', '총인구수']].reset_index(drop=True)
population_csv_not_total_in_seoul_goongoo_total

,행정구역군구,총인구수
0,종로구,140397
1,중구,121441
2,용산구,217288
3,성동구,279374
4,광진구,336169
5,동대문구,338115
6,중랑구,384272
7,성북구,429142
8,강북구,291384
9,도봉구,308986


In [7]:
# 5. 서울의 군구별 인구비율 공공의료기관수 비율을 구한다.
population_vs_hospital = pd.merge(hospital_count_by_goongoo_df, population_csv_not_total_in_seoul_goongoo_total, how = 'outer', on = '행정구역군구')
population_vs_hospital = population_vs_hospital.fillna(0.0)
population_vs_hospital['인구 대비 공공의료기관수 비율'] = (population_vs_hospital['총의료기관수'] / population_vs_hospital['총인구수']) * 100
population_vs_hospital = population_vs_hospital.set_index('행정구역군구')
population_vs_hospital_ratio = population_vs_hospital['인구 대비 공공의료기관수 비율']

In [8]:
# 6. 위 데이터를 활용하여 choropleth map을 작성한다

## 서울시 경계좌표 ##
res = requests.get("https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json")
c = res.content
seoul_geo = json.loads(c)
# seoul_geo

seoul_map = folium.Map(location=[37.5642135, 127.0016985], zoom_start=12)
seoul_map.choropleth(geo_data = seoul_geo, data = population_vs_hospital_ratio, fill_color = 'Reds', 
             fill_opacity = 0.7, line_opacity = 0.2, 
             key_on = 'properties.name', legend_name = '인구수 대비 의료기관비율( % )')
seoul_map

c:\Users\Jinho\anaconda3_it\Lib\site-packages\folium\folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(
